In [ ]:
! pip install -q pyspark==3.2.0 spark-nlp==3.4.2

     |████████████████████████████████| 281.3 MB 31 kB/s 
     |████████████████████████████████| 142 kB 18.3 MB/s 
     |████████████████████████████████| 198 kB 57.0 MB/s 


In [ ]:
import os

In [ ]:
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
# ! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.7.5

import sparknlp

spark = sparknlp.start(spark23=True)

     |████████████████████████████████| 139 kB 32.9 MB/s 


In [ ]:
from pyspark.sql.session import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as f
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, when
from pyspark.sql.functions import udf
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, Word2Vec
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier

In [ ]:
import sparknlp

spark = sparknlp.start(gpu = True, spark23=True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os
spark

In [ ]:
!git clone https://github.com/hoanshiro/Smartphone-ABSA

In [ ]:
%cd /content/Smartphone-ABSA

In [ ]:
train_data = pd.read_csv('data/processed_train.csv')
test_data =  pd.read_csv('data/processed_test.csv')
valid_data =  pd.read_csv('data/processed_valid.csv')

In [ ]:
 train_data = pd.concat([train_data, valid_data], axis=0)

In [ ]:
# Drop N/A
train_sentiment_data = train_data.dropna()
test_sentiment_data = test_data.dropna()

In [ ]:
train_sentiment_data

In [ ]:
#[toxic, obscene]
labels = list(train_data.columns[2:])
def convert2label(row):
  concat_label = ''
  for i in range(len(row)):
    if str(row[i]) in ['1', '1.0']:
      concat_label = concat_label + labels[i] + ', '
  concat_label = concat_label[:-2] + ''
  return concat_label

In [ ]:
train_sentiment_data['labels'] = train_sentiment_data[labels].apply(lambda row: convert2label(row), axis = 1)
test_sentiment_data['labels'] = test_sentiment_data[labels].apply(lambda row: convert2label(row), axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
for df in [train_sentiment_data, test_sentiment_data]:
  df['labels'] = df[labels].apply(lambda row: convert2label(row), axis = 1)

df_train_final = train_sentiment_data[['text', 'labels']]
df_test_final = test_sentiment_data[['text', 'labels']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_train_final

In [ ]:
# Convert Pandas to PySpark DataFrame, train_sentiment_data
Train = spark.createDataFrame(df_train_final)
# Convert Pandas to PySpark DataFrame, test_sentiment_data
Test = spark.createDataFrame(df_test_final) 

In [ ]:
from pyspark.sql.functions import split, col
Train = Train.withColumn('labels', split(col("labels"),","))
#
from pyspark.sql.functions import split, col
Test = Test.withColumn('labels', split(col("labels"),","))

In [ ]:
document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")\
      .setCleanupMode("shrink")

bert_sent = BertSentenceEmbeddings.pretrained("bert_embeddings_bert_base_vi_cased","vi") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

multiClassifier = MultiClassifierDLApproach()\
      .setInputCols("sentence_embeddings")\
      .setOutputCol("category")\
      .setLabelColumn("labels")\
      .setBatchSize(64)\
      .setMaxEpochs(100)\
      .setLr(1e-5)\
      .setThreshold(0.5)\
      .setShufflePerEpoch(True)\
      .setEnableOutputLogs(True)\
      .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        bert_sent,
        multiClassifier
    ])

In [ ]:
! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory


In [ ]:
%%time
pipelineModel = pipeline.fit(Train)

CPU times: user 2.83 s, sys: 311 ms, total: 3.14 s
Wall time: 9min 31s


In [ ]:
preds = pipelineModel.transform(Test)

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
test_preds = preds.select('labels','text',"category.result").toPandas()

In [ ]:
Prediction = spark.createDataFrame(test_preds)
Prediction.show()

In [ ]:
#[toxic, obscene]
labels = list(train_data.columns[2:])
def labels2index(row):
  results = row['result']
  # print(str(results))
  results = [result.replace(' ', '') for result in results]
  for label in results:
    if label != '':
      row[label] = 1
  return row[:-1]

In [ ]:
test = test_preds.apply(lambda row: labels2index(row), axis=1)
test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 2:], test.iloc[:, 2:], target_names=labels))